## Extraindo Embeddings do Pre-trained Bert

In [1]:
!pip install transformers -q

In [2]:
from transformers import BertModel, BertTokenizer
import torch

In [3]:
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [48]:
sentence = "I left my heart on the left side of my chest"

In [49]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
tokens = tokenizer.tokenize(sentence)

In [51]:
print(tokens)

['i', 'left', 'my', 'heart', 'on', 'the', 'left', 'side', 'of', 'my', 'chest']


In [52]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]

In [53]:
tokens = tokens + ["[PAD]"] + ["[PAD]"] + ["[PAD]"]

In [54]:
# attention mask:
attention_mask = [1 if i!="[PAD]" else 0 for i in tokens]
attention_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

In [55]:
# mapping all the tokens to a unique token ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[101,
 1045,
 2187,
 2026,
 2540,
 2006,
 1996,
 2187,
 2217,
 1997,
 2026,
 3108,
 102,
 0,
 0,
 0]

In [56]:
# convert to a pytorch tensor, which can be used as an input to neural network
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [57]:
output = model(token_ids, attention_mask= attention_mask)

In [58]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0714,  0.1934,  0.0518,  ..., -0.0147,  0.1046,  0.2315],
         [ 0.3465, -0.0633, -0.2773,  ..., -0.1543,  0.6000,  0.3067],
         [ 0.8287, -0.3581, -0.1578,  ...,  0.2077, -0.1536, -0.2053],
         ...,
         [-0.4109, -0.1541,  0.2781,  ...,  0.6018,  0.2186,  0.0985],
         [-0.4033, -0.1575,  0.2736,  ...,  0.6098,  0.2491, -0.0096],
         [-0.2461, -0.0954,  0.2431,  ...,  0.5231,  0.0994,  0.0503]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8063, -0.1587,  0.1560,  0.5503, -0.0806, -0.0649,  0.7011,  0.0358,
          0.1782, -0.9997,  0.2340,  0.2218,  0.9670, -0.1527,  0.8780, -0.3196,
          0.1545, -0.4318,  0.2172, -0.3763,  0.4937,  0.9429,  0.5706,  0.0523,
          0.2858,  0.4339, -0.3908,  0.8844,  0.9275,  0.6095, -0.4046,  0.0610,
         -0.9775, -0.0217,  0.1211, -0.9714,  0.0509, -0.6692,  0.1164,  0.1250,
         -0.8412,  0.1165,  0.99

In [43]:
import numpy as np

In [46]:
norm_left1 = torch.matmul(output["last_hidden_state"][0][2], output["last_hidden_state"][0][2])
norm_left2 = torch.matmul(output["last_hidden_state"][0][7], output["last_hidden_state"][0][7])

norm_my1 = torch.matmul(output["last_hidden_state"][0][3], output["last_hidden_state"][0][3])
norm_my2 = torch.matmul(output["last_hidden_state"][0][10], output["last_hidden_state"][0][10])

print(np.sqrt(norm_left1.detach().numpy()),
np.sqrt(norm_left2.detach().numpy()),
np.sqrt(norm_my1.detach().numpy()),
np.sqrt(norm_my2.detach().numpy()))

14.522059 15.587973 14.792152 14.642604


In [59]:
norm_left1 = torch.matmul(output["last_hidden_state"][0][2], output["last_hidden_state"][0][2])
norm_left2 = torch.matmul(output["last_hidden_state"][0][7], output["last_hidden_state"][0][7])

norm_my1 = torch.matmul(output["last_hidden_state"][0][3], output["last_hidden_state"][0][3])
norm_my2 = torch.matmul(output["last_hidden_state"][0][10], output["last_hidden_state"][0][10])

print(np.sqrt(norm_left1.detach().numpy()),
np.sqrt(norm_left2.detach().numpy()),
np.sqrt(norm_my1.detach().numpy()),
np.sqrt(norm_my2.detach().numpy()))

12.864475 14.44965 13.149075 12.719797


In [47]:
dot_left = torch.matmul(output["last_hidden_state"][0][2], output["last_hidden_state"][0][7])
print(f'produto interno entre last: {dot_left}')
# 0,5476


dot_my = torch.matmul(output["last_hidden_state"][0][3], output["last_hidden_state"][0][10])
print(f'produto interno entre my: {dot_my}')
# 0,9

produto interno entre last: 115.3312759399414
produto interno entre my: 177.2930908203125


In [61]:
dot_left = torch.matmul(output["last_hidden_state"][0][2], output["last_hidden_state"][0][7])/ (np.sqrt(norm_left1.detach().numpy()) * np.sqrt(norm_left2.detach().numpy()))
print(f'produto interno entre last: {dot_left}')


dot_my = torch.matmul(output["last_hidden_state"][0][3], output["last_hidden_state"][0][10])/ (np.sqrt(norm_my1.detach().numpy()) * np.sqrt(norm_my2.detach().numpy()))
print(f'produto interno entre my: {dot_my}')


produto interno entre last: 0.5635417103767395
produto interno entre my: 0.8812980651855469
